### Solutions for 02-Working_with_ASDF_Files

**Exercise 1:**

In [1]:
import asdf

**Read the file and use the `info` method to look at the contents**.

`info` has arguments which control the behavior. Use the defaults to look at the contents of the file.

In [2]:
af = asdf.open('../data/r0000101001001001001_01101_0001_WFI01_cal.asdf')

In [3]:
af.info(max_rows=200)

root (AsdfObject)
├─asdf_library (Software)
│ ├─author (str): The ASDF Developers
│ ├─homepage (str): http://github.com/asdf-format/asdf
│ ├─name (str): asdf
│ └─version (str): 2.11.2.dev13+gf9aeb247
├─history (dict)
│ └─extensions (list)
│   ├─[0] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/core/extensions/core-1.5.0
│   │ └─software (Software) ...
│   ├─[1] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/astronomy/coordinates/extensions/coordinates-1.0.0
│   │ └─software (Software) ...
│   ├─[2] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension.BuiltinExtension
│   │ └─software (Software) ...
│   ├─[3] (ExtensionMetadata)
│   │ ├─extension_class (str): asdf.extension._manifest.ManifestExtension
│   │ ├─extension_uri (str): asdf://asdf-format.org/astronomy/gwcs/extensions/

In [4]:
af.search('wcs')

root (AsdfObject)
└─roman (WfiImage) # The schema for WFI Level 2 images.

  └─meta (dict)
    ├─cal_step (CalStep) # Calibration Status
    │ └─assign_wcs (str): COMPLETE # Assign World Coordinate System
    ├─wcs (WCS)
    └─wcsinfo (Wcsinfo) # WCS parameters

In [5]:
w = af['roman']['meta']['wcs']
print(w)

  From     Transform  
-------- -------------
detector CompoundModel
    v2v3      v23tosky
   world          None


In [6]:
ra, dec = w(200, 300)
print(ra, dec)

84.69946280125302 -69.11808059517361


In [7]:
sky = w.pixel_to_world(200, 300)
print(sky)

<SkyCoord (ICRS): (ra, dec) in deg
    (84.6994628, -69.1180806)>


In [8]:
af['roman']['data']

array([[ 3.5502228e+01,  3.0198467e+01,  1.1236250e+01, ...,
         1.5567736e+01,  3.4690548e+01,  4.5890507e+01],
       [ 2.4822083e+01,  2.0766781e+01,  5.4226227e+01, ...,
         2.5411541e+01,  1.9779814e+01,  3.4319172e+00],
       [ 2.3591185e+01,  3.6522991e+01,  2.7500755e+01, ...,
         6.9299988e+01,  1.8446072e+01,  2.7945328e+01],
       ...,
       [-5.0138377e-02,  4.8151932e+00,  4.0247574e+01, ...,
         3.9501991e+01,  5.0384731e+00,  2.2977093e+01],
       [ 1.8236900e+01,  1.6879545e+01,  2.0073387e+01, ...,
         1.3043494e+02,  2.8485823e+01,  5.6190315e+01],
       [ 2.2255131e+01,  2.5889729e+01,  4.6622475e+01, ...,
        -1.0151666e+00,  4.0005136e+00,  6.9930313e+01]], dtype=float32)

In [9]:
#af['roman']['data'][0, 0] = 999

**Exercise 2:**

Add `additionalProperties=false` to the schema and open the file

In [10]:
s = """
%YAML 1.1
---
$schema: http://stsci.edu/schemas/yaml-schema/draft-01

title: Mickey's pet
description: |
  Basic info and a picture of Mickie's 
  dog Pluto.

type: object
properties:
  age:
    title: The age of Pluto
    type: object
    properties:
      birthday:
        title: Pluto's first showing
        tag: tag:stsci.edu:asdf/time/time-1.1.0
  mass:
    title: How much he weighs.
    tag: tag:stsci.edu:asdf/unit/quantity-1.1.0
  picture:
    tag: tag:stsci.edu:asdf/core/ndarray-1.0.0
  name:
    title: Name
    type: string
required: [name, picture]
additionalProperties: false
...
"""

In [11]:
f = open('add-prop-1.0.0.yaml', mode='w')
f.write(s)
f.close()

In [12]:
afs = asdf.open('pluto.asdf', custom_schema='./add-prop-1.0.0.yaml')

ValidationError: Additional properties are not allowed ('asdf_library', 'birthday', 'history' were unexpected)

Failed validating 'additionalProperties' in schema:
    {'$schema': 'http://stsci.edu/schemas/yaml-schema/draft-01',
     'additionalProperties': False,
     'description': "Basic info and a picture of Mickie's \ndog Pluto.\n",
     'properties': {'age': {'properties': {'birthday': {'tag': 'tag:stsci.edu:asdf/time/time-1.1.0',
                                                        'title': "Pluto's "
                                                                 'first '
                                                                 'showing'}},
                            'title': 'The age of Pluto',
                            'type': 'object'},
                    'mass': {'tag': 'tag:stsci.edu:asdf/unit/quantity-1.1.0',
                             'title': 'How much he weighs.'},
                    'name': {'title': 'Name', 'type': 'string'},
                    'p

Take the original schema and add a new required property, called `friend`.

In [ ]:
s = """
%YAML 1.1
---
$schema: http://stsci.edu/schemas/yaml-schema/draft-01

title: Mickey's pet
description: |
  Basic info and a picture of Mickie's 
  dog Pluto.

type: object
properties:
  age:
    title: The age of Pluto
    type: object
    properties:
      birthday:
        title: Pluto's first showing
        tag: tag:stsci.edu:asdf/time/time-1.1.0
  mass:
    title: How much he weighs.
    tag: tag:stsci.edu:asdf/unit/quantity-1.1.0
  picture:
    tag: tag:stsci.edu:asdf/core/ndarray-1.0.0
  name:
    title: Name
    type: string
  friend:
    type: string
    title: "Who is Pluto's friend?"
required: [name, picture, friend]
...
"""

In [ ]:
f = open('pluto-friend-1.0.0.yaml', mode='w')
f.write(s)
f.close()

In [ ]:
afs = asdf.open('pluto.asdf', custom_schema='./pluto-friend-1.0.0.yaml')

In [ ]:
asf = asdf.open('pluto.asdf', mode='rw')
asf['friend'] = 'Mickey'
#asf.update()
asf.write_to('pluto-friend.asdf')

In [ ]:
asf = asdf.open('pluto-friend.asdf', custom_schema='./pluto-friend-1.0.0.yaml')